In [1]:
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import pyabf
import numpy as np
import pandas as pd
from collections import Counter
from scipy.signal import savgol_filter

#Written by Danny Lim
#Updated 5/11/2023

In [25]:
def abf_zc(mypath, if_plot=False):
    abf = pyabf.ABF(mypath)
    abf.setSweep(sweepNumber= 0, channel=0)
    pY, pX = savgol_filter(abf.sweepY, window_length=5, polyorder=2), abf.sweepX
    abf.setSweep(sweepNumber= 0, channel=1)
    tY, tX = savgol_filter(abf.sweepY, window_length=5, polyorder=2), abf.sweepX

    if if_plot:
        fig, ax = plt.subplots(figsize=(30,5))
        ax1 = ax.plot(tX, tY, label = 'stim') #tY = stim
        ax2 = ax.plot(pX, pY, label = 'y-voltage') #pY = y-voltage
        ax.legend(['stim','y-voltage'])
        plt.show()

    stimV = tY
    frameV = pY

    frameTimes = np.where(np.diff(np.sign(frameV)==1)!=0)[0][::2]

    a = stimV - (0.5 * stimV.max())
    zeroCrossings = np.where(np.diff(np.sign(a)==1)!=0)[0]
    stimTimes = zeroCrossings[::2]

    stimFrameVector = np.empty(len(stimTimes))
    for s in range(len(stimTimes)):
        gotime = stimTimes[s]
        a = frameTimes > gotime
        stimFrameVector[s] = np.where(a)[0][0]-1

    stimMarkerTrace = 0. * frameTimes
    for f in range(len(frameTimes)):
        stimMarkerTrace[f] = stimV[frameTimes[f]]
    framestamp = stimMarkerTrace > 1
    zeroCrossings = np.where(np.diff(np.sign(framestamp.astype("int"))==1))[0]
    
    return zeroCrossings, len(pY)//np.diff(frameTimes)[0]

In [30]:
def abfs(mypath):
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) if f.split(".")[-1] == "abf" ]

    abf_list = sorted(onlyfiles)
    print("processing files in this order... : ")
    print(abf_list)

    zeroCrossings = []
    l = 0
    for abf_file in abf_list:
        zc, last_frame = abf_zc(mypath + '/' + abf_file)
        zeroCrossings = zeroCrossings + list(zc+l+1) # +1 because python index starts from 0
        l += last_frame
        #print(zc, last_frame)
        
    np.save(mypath + "ZeroCrossings.npy",zeroCrossings)
    return np.array(zeroCrossings)

In [31]:
mypath = "/Volumes/My Passport for Mac/SampleData_/230214/LA24_1/abf"
zeroCrossings = abfs(mypath)
zeroCrossings

processing files in this order... : 
['23214000.abf']


array([ 401,  491,  530, ..., 8492, 8531, 8621])